# Implementing the Graph Autoencoder (GAE) and Struc2vec methodologies on the Graphics Processing Unit (GPU) #

As delineated in the `Protein_Embedding.ipynb` file, we aim to employ the `BioNEV` repository from *GitHub* to derive protein embeddings for our dataset.

Following the cloning of the repository, we conducted specific code adjustments to accommodate our version of Python, TensorFlow, Numpy, and other relevant libraries.

To achieve accelerated processing times, the utilization of a Graphics Processing Unit (GPU) is imperative as opposed to relying solely on the Central Processing Unit (CPU).

 For this purpose, we have opted to employ the `Tesla T4` GPU.


In [1]:
!nvidia-smi

Fri Nov 17 08:47:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downlading the right Tensorflow version ##

To address certain issues encountered with TensorFlow version 2.14.0, we have decided to revert to the preceding version, specifically version 2.13.0.

In [2]:
!pip install tensorflow==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 42.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.

### Verify the version of Tensorflow ###

The next step is to make sure `Tensorflow` vesion is `2.13.0`

In [3]:
import tensorflow as tf

print(tf.__version__)

2.13.0


## Installing `tensorrt` ##

The subsequent phase involves the installation of the TensorRT library.

In [5]:
!python -m pip install --upgrade tensorrt

  Using cached tensorrt-8.6.1.post1.tar.gz (18 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=72396fc0ebcc44fd9fa2f16cf5eca9eb0009238401c90c1d4e2c9792f4887bac
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


## Verify the usage of GPU ##

The next step if to verify that `Tensorflow` uses the avilable `GPU`

In [6]:
!python3 -c "import tensorrt as trt;import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Executing GAE and Struc2Vec methods ##

Once we have acquired the appropriate libraries and confirmed the functionality of GPU utilization, we can proceed to formulate the requisite methodologies for generating protein embeddings.

In [7]:
cd "/content/drive/MyDrive/DeepLearning Research/BioNEV"

/content/drive/MyDrive/DeepLearning Research/BioNEV


In [8]:
!pip install -e.

Obtaining file:///content/drive/MyDrive/DeepLearning%20Research/BioNEV
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512407 sha256=23bf6bd58c70a863cf30e6956789bbb59a31552f14bc6fb8e5cfb7b915b9726e
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw
  Running setup.py develop for bionev


In [9]:
!pwd

/content/drive/MyDrive/DeepLearning Research/BioNEV


In [10]:
print(tf.test.gpu_device_name())

/device:GPU:0


### Executing GAE method ###

In [12]:
!bionev --input ./data/STRING_PPI_11_0_.edgelist \
       --output ./embeddings/STRING_PPI_11_0_GAE_embeddings.txt \
       --dimensions 500 \
       --method GAE \
       --epochs 800 \
       --task link-prediction \
       --eval-result-file eval_result.txt

2023-11-17 09:03:54.376630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
######################################################################
Embedding Method: GAE, Evaluation Task: link-prediction
######################################################################
Original Graph: nodes: 16063 edges: 389682
Training Graph: nodes: 16063 edges: 312155
Loading training graph for learning embedding...
Graph Loaded...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
targets is deprecated, use labels instead
2023-11-17 09:04:01.891867: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Epoch: 0001 train_loss= 0.70204 train_acc= 0.00250 time= 3.18059
Epoch: 0101 train_loss= 0.44972 train_acc= 0.47437 time= 0.658

## Executing struc2vec method ##

In [13]:
!CUDA_VISIBLE_DEVICES=0 bionev --input ./data/STRING_PPI_11_0_.edgelist \
     --output ./embeddings/STRING_PPI_11_0_s2vec_embeddings.txt \
     --dimensions 500 \
     --method struc2vec \
     --task link-prediction \
     --eval-result-file eval_result.txt

2023-11-17 09:26:29.212053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
######################################################################
Embedding Method: struc2vec, Evaluation Task: link-prediction
######################################################################
Original Graph: nodes: 16063 edges: 389682
Training Graph: nodes: 16063 edges: 312155
Loading training graph for learning embedding...
Graph Loaded...
create distances network..
rm: cannot remove '/content/drive/MyDrive/DeepLearning': No such file or directory
rm: cannot remove 'Research/BioNEV/src/bionev/struc2vec/pickles/weights_distances-layer-*.pickle': No such file or directory
rm: cannot remove '/content/drive/MyDrive/DeepLearning': No such file or directory
rm: cannot remove 'Research/BioNEV/src/bionev/struc2vec/pickles/graphs-layer-*.pickle': No such file or directory
rm: cannot remove '/content/drive/MyDrive/DeepLearning': No such file or directory
rm: ca